In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import html
import preprocessor as pp
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

from nltk.probability import FreqDist

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manasip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/manasip/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
##Import file
df_gen = pd.read_csv("df_gender.csv")

In [46]:
df_gen.head(3)

,Tweet Id,Text,Username,Noisy,Location,Name,User_Id,Year,sentiment,sentimentclass,sentimentnum,Gender
0,9.472559e+17,I just read ur twitter comments on ur acct u h...,kathy_tkat,abortion,"Massachusetts, USA",Kathy,7.266530e+17,2017,"{'neg': 0.046, 'neu': 0.86, 'pos': 0.093, 'com...",positive,2,female
1,9.472487e+17,I live in a country where abortion is illegal ...,belindacree,abortion,Northern Ireland,Belinda,1.393754e+08,2017,"{'neg': 0.27, 'neu': 0.669, 'pos': 0.061, 'com...",negative,1,female
2,9.472429e+17,If the administration manages to outlaw aborti...,Bouteloua_spp,abortion,None of your god damn business,Michelle,2.054931e+08,2017,"{'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'comp...",negative,1,female


In [47]:
df_gen.rename(columns={"Tweet Id": "id", "Text": "tweet", "Username": "user", \
                       "Gender":"gender", "Noisy":"noisy"}, \
                   inplace=True)

In [48]:
##Sanity check
print(df_gen.id.nunique(), df_gen.shape[0])

8584 8584


In [49]:
df_gen.drop_duplicates(subset=['id'], inplace=True)

In [50]:
#Sanity check
df_gen.shape

(8584, 12)

------------Pre-processing tweets------------

In [51]:
df_gen["clean_tweet"] = df_gen.tweet

In [52]:
#Step 1: Decode html escape characters: &amp &quote etc 
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))

In [53]:
#Sanity check
df_gen['check'] = df_gen.clean_tweet.str.contains("&", case=False, na=False).astype(int) 
# Checked for &gt, &amp &#62 &quote
print(sum(df_gen.check))

0


In [54]:
#Step 2: Remove @mentions
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: re.sub('@[^\s]+','',x))
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: re.sub('@[^\w]+','',x))
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: re.sub('@','',x))

In [55]:
#Sanity check
df_gen.check = df_gen.clean_tweet.str.contains("@", case=False, na=False).astype(int) 
print(sum(df_gen.check))

0


In [56]:
#Step 3 Remove \n \t \r
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: x.replace('\n', '').replace('\t', '').replace('\r', ''))

In [57]:
#Sanity check
df_gen.check = df_gen.clean_tweet.str.contains("\n", case=False, na=False).astype(int) 
print('n', sum(df_gen.check))

df_gen.check = df_gen.clean_tweet.str.contains("\t", case=False, na=False).astype(int) 
print('t', sum(df_gen.check))

df_gen.check = df_gen.clean_tweet.str.contains("\r", case=False, na=False).astype(int) 
print('r', sum(df_gen.check))

n 0
t 0
r 0


In [58]:
#Step 4: Tweet-preprocessor to remove, urls, emojis, hashtags, reserved words, mentions (if any), smileys
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: pp.clean(x))

In [59]:
#Sanity check
df_gen.check = df_gen.clean_tweet.str.contains("#", case=False, na=False).astype(int) 
print("check #",sum(df_gen.check))
df_gen.check = df_gen.clean_tweet.str.contains("http", case=False, na=False).astype(int) 
print("check http",sum(df_gen.check))
df_gen.check = df_gen.clean_tweet.str.contains("http", case=False, na=False).astype(int) 
print("check www",sum(df_gen.check))

check # 0
check http 0
check www 0


In [60]:
#Step 5: Remove Placeholders
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: re.sub(r'{link}', '', x))
df_gen.clean_tweet = df_gen.clean_tweet.apply(lambda x: re.sub(r"\[video\]", '', x))

In [61]:
#Sanity check
df_gen.check = df_gen.clean_tweet.str.contains("{link}", case=False, na=False).astype(int) 
print("check link",sum(df_gen.check))
df_gen.check = df_gen.clean_tweet.str.contains("\[video\]", case=False, na=False).astype(int) 
print("check video",sum(df_gen.check))

check link 0
check video 0


In [62]:
#Step 6: Remove punctuations, digits
#Create helper function
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def remove_digits(text):
    for digit in string.digits:
        text = text.replace(digit, '')
    return text

df_gen.clean_tweet = df_gen.clean_tweet.apply(remove_punctuations)

df_gen.clean_tweet = df_gen.clean_tweet.apply(remove_digits)

In [63]:
#Sanity check
df_gen.check = df_gen.clean_tweet.str.contains("\!", case=False, na=False).astype(int)
print("check !",sum(df_gen.check))
df_gen.check = df_gen.clean_tweet.str.contains("\?", case=False, na=False).astype(int)
print("check ?",sum(df_gen.check))
df_gen.check = df_gen.clean_tweet.str.contains("\.", case=False, na=False).astype(int)
print("check .",sum(df_gen.check))

df_gen.check = df_gen.clean_tweet.str.contains("1", case=False, na=False).astype(int)
print("check 1",sum(df_gen.check))

check ! 0
check ? 0
check . 0
check 1 0


In [64]:
#Step 7 Lowercase before tokenization
df_gen.clean_tweet = df_gen.clean_tweet.str.lower()

In [65]:
#Step 8 Tokenization
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

In [66]:
df_gen["tok_tweet"] = df_gen.clean_tweet.apply(lambda x: tknzr.tokenize(x))

In [67]:
#Step 8 Remove stopwords 
#Create helper function

stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

df_gen.tok_tweet = df_gen.tok_tweet.apply(lambda x: remove_stopwords(x))

In [68]:
#Step 9 Lemmatization
#Create helper function

lem = nltk.WordNetLemmatizer()

def lemmat(text):
    text = [lem.lemmatize(word) for word in text]
    return text

df_gen.tok_tweet = df_gen.tok_tweet.apply(lambda x: lemmat(x))

In [69]:
#Drop words less than 3 words
tokens3 = df_gen[df_gen.tok_tweet.apply(lambda x: len(x) <= 3)].index

In [70]:
df_gen.drop(tokens3, inplace = True)
df_gen.reset_index(inplace=True)

In [71]:
all_words = [word for tokens in df_gen.tok_tweet for word in tokens]
tweet_lengths = [len(tokens) for tokens in df_gen.tok_tweet]
vocab = sorted(list(set(all_words)))

print('{} tokens total, with a vocabulary size of {}'.format(len(all_words), len(vocab)))
print('Max tweet length is {}'.format(max(tweet_lengths)))

108246 tokens total, with a vocabulary size of 10996
Max tweet length is 39


In [72]:
df_gen.head()

,index,id,tweet,user,noisy,Location,Name,User_Id,Year,sentiment,sentimentclass,sentimentnum,gender,clean_tweet,check,tok_tweet
0,0,9.472559e+17,I just read ur twitter comments on ur acct u h...,kathy_tkat,abortion,"Massachusetts, USA",Kathy,7.266530e+17,2017,"{'neg': 0.046, 'neu': 0.86, 'pos': 0.093, 'com...",positive,2,female,i just read ur twitter comments on ur acct u h...,0,"[read, ur, twitter, comment, ur, acct, u, huge..."
1,1,9.472487e+17,I live in a country where abortion is illegal ...,belindacree,abortion,Northern Ireland,Belinda,1.393754e+08,2017,"{'neg': 0.27, 'neu': 0.669, 'pos': 0.061, 'com...",negative,1,female,i live in a country where abortion is illegal ...,0,"[live, country, abortion, illegal, woman, trag..."
2,2,9.472429e+17,If the administration manages to outlaw aborti...,Bouteloua_spp,abortion,None of your god damn business,Michelle,2.054931e+08,2017,"{'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'comp...",negative,1,female,if the administration manages to outlaw aborti...,0,"[administration, manages, outlaw, abortion, go..."
3,3,9.472395e+17,I dont put my hat in the abortion discussion m...,DerekTheBard,abortion,"Elmira, NY",Derek,1.539154e+09,2017,"{'neg': 0.238, 'neu': 0.71, 'pos': 0.051, 'com...",negative,1,male,i dont put my hat in the abortion discussion m...,0,"[dont, put, hat, abortion, discussion, much, t..."
4,4,9.472300e+17,To those saying Life begins at conception I sa...,Deborahtaxi,abortion,"New York, USA",Debbie,1.434785e+08,2017,"{'neg': 0.044, 'neu': 0.785, 'pos': 0.17, 'com...",positive,2,female,to those saying life begins at conception i sa...,0,"[saying, life, begin, conception, say, take, c..."


-------------LDA Modelling--------------------

In [78]:
#Create 4 dataframes
#Profile spilt by male and female
df_lf = df_gen[(df_gen.noisy=="prolife") & (df_gen.gender=="female")]
df_lm = df_gen[(df_gen.noisy=="prolife") & (df_gen.gender=="male")]

#Prochoice spilt by male and female
df_cf = df_gen[(df_gen.noisy=="prochoice") & (df_gen.gender=="female")]
df_cm = df_gen[(df_gen.noisy=="prochoice") & (df_gen.gender=="male")]

In [79]:
##Sanity check
print("prolife female: ", "gender:", df_lf.gender.unique(), "label:", df_lf.noisy.unique())
print("prolife male: ", "gender:", df_lm.gender.unique(), "label:", df_lm.noisy.unique())
print("prochoice female: ", "gender:", df_cf.gender.unique(), "label:", df_cf.noisy.unique())
print("prochoice male: ", "gender:", df_cf.gender.unique(), "label:", df_cf.noisy.unique())

prolife female:  gender: ['female'] label: ['prolife']
prolife male:  gender: ['male'] label: ['prolife']
prochoice female:  gender: ['female'] label: ['prochoice']
prochoice male:  gender: ['female'] label: ['prochoice']


In [80]:
#Create dictionary
txt_dict_lf = Dictionary(df_lf.tok_tweet)
txt_dict_lm = Dictionary(df_lm.tok_tweet)
txt_dict_cf = Dictionary(df_cf.tok_tweet)
txt_dict_cm = Dictionary(df_cm.tok_tweet)

In [81]:
txt_out_lf = txt_dict_lf.token2id
txt_out_lm = txt_dict_lm.token2id
txt_out_cf = txt_dict_cf.token2id
txt_out_cm = txt_dict_cm.token2id

In [82]:
#Create BOW
tweets_bow_lf = [txt_dict_lf.doc2bow(tweet) for tweet in df_lf.tok_tweet]
tweets_bow_lm = [txt_dict_lm.doc2bow(tweet) for tweet in df_lm.tok_tweet]
tweets_bow_cf = [txt_dict_cf.doc2bow(tweet) for tweet in df_cf.tok_tweet]
tweets_bow_cm = [txt_dict_cm.doc2bow(tweet) for tweet in df_cm.tok_tweet]

In [86]:
##Model
#Female Prolife
k = 5
tweets_lda_lf = LdaModel(tweets_bow_lf,
                      num_topics = k,
                      id2word = txt_dict_lf,
                      random_state = 10,
                      passes=100)

In [87]:
tweets_lda_lf.show_topics()

[(0,
  '0.099*"pro" + 0.085*"life" + 0.012*"child" + 0.011*"prolife" + 0.011*"people" + 0.010*"woman" + 0.009*"birth" + 0.009*"choice" + 0.008*"abortion" + 0.008*"baby"'),
 (1,
  '0.023*"prolife" + 0.014*"life" + 0.014*"pro" + 0.012*"birth" + 0.010*"woman" + 0.009*"child" + 0.009*"healthcare" + 0.009*"cut" + 0.008*"abortion" + 0.006*"food"'),
 (2,
  '0.047*"prolife" + 0.011*"abortion" + 0.008*"people" + 0.008*"amp" + 0.008*"vote" + 0.007*"baby" + 0.007*"would" + 0.006*"life" + 0.006*"see" + 0.005*"trump"'),
 (3,
  '0.062*"prolife" + 0.019*"abortion" + 0.014*"amp" + 0.012*"right" + 0.011*"life" + 0.011*"woman" + 0.007*"human" + 0.006*"adoption" + 0.006*"issue" + 0.005*"defundpp"'),
 (4,
  '0.044*"prolife" + 0.024*"life" + 0.022*"pro" + 0.012*"amp" + 0.010*"vote" + 0.007*"child" + 0.007*"would" + 0.007*"u" + 0.005*"trump" + 0.005*"good"')]

In [88]:
from gensim.models.coherencemodel import CoherenceModel

In [92]:
cm = CoherenceModel(model=tweets_lda_lf, texts=df_lf.tok_tweet.to_list(), coherence='c_v')
cm.get_coherence()

0.3159526015867854

In [93]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

/Users/manasip/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [94]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(tweets_lda_lf, tweets_bow_lf, dictionary=txt_dict_lf)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.153207 -0.043308       1        1  45.700896
4     -0.007888 -0.042643       2        1  16.704317
3      0.115511 -0.017321       3        1  14.455259
2      0.065933 -0.033251       4        1  12.325401
1     -0.020350  0.136523       5        1  10.814127, topic_info=         Term         Freq        Total Category  logprob  loglift
7         pro  1121.000000  1121.000000  Default  30.0000  30.0000
26    prolife   659.000000   659.000000  Default  29.0000  29.0000
47      birth   130.000000   130.000000  Default  28.0000  28.0000
6        life  1034.000000  1034.000000  Default  27.0000  27.0000
21   abortion   201.000000   201.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
136       get     8.438677    74.332780   Topic5  -5.6448   0.0486
28          u     8.232035    72.090645   Topic5  -5.6696   0.0544
215      want     8.496556   100.313742   Topic5  -5.6380  -0.2443
248      like     8.083358    75.366808   Topic5  -5.6879  -0.0082
504       gop     7.665404    67.666811   Topic5  -5.7409   0.0464

[368 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
21        1  0.391730  abortion
21        2  0.074379  abortion
21        3  0.297516  abortion
21        4  0.143800  abortion
21        5  0.094213  abortion
...     ...       ...       ...
301       1  0.926882      yeah
3516      1  0.975734     yemen
1879      1  0.940581     youre
1879      2  0.024117     youre
1879      5  0.048235     youre

[655 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 2])

In [95]:
##Model
#Male Prolife
k = 5
tweets_lda_lm = LdaModel(tweets_bow_lm,
                      num_topics = k,
                      id2word = txt_dict_lm,
                      random_state = 10,
                      passes=100)

In [96]:
tweets_lda_lm.show_topics()

[(0,
  '0.059*"pro" + 0.050*"life" + 0.024*"prolife" + 0.014*"conservative" + 0.014*"abortion" + 0.012*"trump" + 0.011*"justice" + 0.010*"marriage" + 0.010*"late" + 0.009*"pray"'),
 (1,
  '0.065*"pro" + 0.058*"life" + 0.024*"prolife" + 0.011*"people" + 0.011*"choice" + 0.011*"praytoendabortion" + 0.009*"birth" + 0.009*"day" + 0.008*"born" + 0.007*"tcot"'),
 (2,
  '0.049*"prolife" + 0.011*"u" + 0.010*"holylove" + 0.009*"unitedhearts" + 0.009*"ourlady" + 0.009*"guadalupe" + 0.009*"apparition" + 0.009*"feast" + 0.009*"praytoendabortion" + 0.008*"maranathasns"'),
 (3,
  '0.047*"prolife" + 0.023*"abortion" + 0.020*"life" + 0.013*"pro" + 0.011*"alert" + 0.010*"woman" + 0.008*"ohio" + 0.005*"bill" + 0.005*"sign" + 0.005*"via"'),
 (4,
  '0.040*"life" + 0.039*"prolife" + 0.038*"pro" + 0.014*"human" + 0.012*"child" + 0.011*"right" + 0.009*"unitedhearts" + 0.009*"baby" + 0.009*"vote" + 0.008*"jesus"')]

In [97]:
cm = CoherenceModel(model=tweets_lda_lm, texts=df_lm.tok_tweet.to_list(), coherence='c_v')
cm.get_coherence()

0.3776528707315047

In [98]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(tweets_lda_lm, tweets_bow_lm, dictionary=txt_dict_lm)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.049606 -0.122232       1        1  25.656217
0      0.105289  0.026258       2        1  23.492399
1      0.073140  0.017400       3        1  22.855247
3     -0.039885  0.113377       4        1  15.703610
2     -0.188150 -0.034803       5        1  12.292526, topic_info=         Term        Freq       Total Category  logprob  loglift
10        pro  813.000000  813.000000  Default  30.0000  30.0000
8        life  774.000000  774.000000  Default  29.0000  29.0000
21    prolife  692.000000  692.000000  Default  28.0000  28.0000
184   justice   50.000000   50.000000  Default  27.0000  27.0000
185      late   46.000000   46.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
31       make   13.139058   53.664565   Topic5  -5.2392   0.6890
11      right   16.129080  117.100175   Topic5  -5.0342   0.1138
572       say   13.239751   67.737612   Topic5  -5.2316   0.4638
115      tcot   11.831029   47.582592   Topic5  -5.3441   0.7044
15   abortion   14.230595  201.669136   Topic5  -5.1594  -0.5551

[317 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
15        1  0.178510  abortion
15        2  0.322310  abortion
15        3  0.074379  abortion
15        4  0.352062  abortion
15        5  0.069421  abortion
...     ...       ...       ...
1487      1  0.108484     youre
1487      2  0.253129     youre
1487      3  0.253129     youre
1487      4  0.397775     youre
1746      1  0.936882  zootopia

[514 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])

In [99]:
##Model
#Female Prochoice
k = 5
tweets_lda_cf = LdaModel(tweets_bow_cf,
                      num_topics = k,
                      id2word = txt_dict_cf,
                      random_state = 10,
                      passes=100)

In [100]:
tweets_lda_cf.show_topics()

[(0,
  '0.062*"prochoice" + 0.017*"abortion" + 0.015*"child" + 0.013*"life" + 0.012*"amp" + 0.012*"prolife" + 0.011*"pro" + 0.010*"want" + 0.008*"people" + 0.007*"woman"'),
 (1,
  '0.067*"prochoice" + 0.025*"abortion" + 0.017*"woman" + 0.006*"right" + 0.006*"amp" + 0.006*"u" + 0.006*"life" + 0.006*"control" + 0.005*"prolife" + 0.004*"support"'),
 (2,
  '0.082*"prochoice" + 0.029*"woman" + 0.018*"abortion" + 0.018*"choice" + 0.015*"body" + 0.013*"right" + 0.010*"pro" + 0.010*"prolife" + 0.008*"make" + 0.007*"amp"'),
 (3,
  '0.058*"prochoice" + 0.017*"one" + 0.015*"abortion" + 0.010*"prolife" + 0.008*"like" + 0.007*"nation" + 0.007*"voteprochoice" + 0.006*"vote" + 0.006*"let" + 0.005*"must"'),
 (4,
  '0.068*"prochoice" + 0.017*"abortion" + 0.013*"woman" + 0.010*"right" + 0.008*"people" + 0.007*"need" + 0.007*"get" + 0.006*"womensrights" + 0.006*"amp" + 0.006*"v"')]

In [101]:
cm = CoherenceModel(model=tweets_lda_cf, texts=df_cf.tok_tweet.to_list(), coherence='c_v')
cm.get_coherence()

0.30972257339213

In [102]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(tweets_lda_cf, tweets_bow_cf, dictionary=txt_dict_cf)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.074434  0.054205       1        1  42.850310
0      0.055905 -0.007032       2        1  16.411588
1      0.029029 -0.010912       3        1  16.346623
4     -0.045296 -0.108485       4        1  15.568891
3     -0.114073  0.072224       5        1   8.822587, topic_info=          Term         Freq        Total Category  logprob  loglift
5    prochoice  1389.000000  1389.000000  Default  30.0000  30.0000
22        body   122.000000   122.000000  Default  29.0000  29.0000
203        one    87.000000    87.000000  Default  28.0000  28.0000
14      choice   160.000000   160.000000  Default  27.0000  27.0000
13       child    60.000000    60.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
308        see     4.580790    32.172489   Topic5  -5.9243   0.4786
179     people     5.464235    89.312315   Topic5  -5.7479  -0.3661
33        life     5.413334   133.404557   Topic5  -5.7573  -0.7767
16         get     4.998405    93.558322   Topic5  -5.8370  -0.5016
276       time     4.494244    47.548318   Topic5  -5.9434   0.0689

[378 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
356       1  0.888604      able
356       4  0.111075      able
9         1  0.416031  abortion
9         2  0.146998  abortion
9         3  0.219110  abortion
...     ...       ...       ...
616       4  0.122714       yes
616       5  0.040905       yes
566       1  0.273352        yo
566       3  0.683380        yo
3416      2  0.939388     youll

[622 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 5, 4])

In [103]:
##Model
#Male Prochoice
k = 5
tweets_lda_cm = LdaModel(tweets_bow_cm,
                      num_topics = k,
                      id2word = txt_dict_cm,
                      random_state = 10,
                      passes=100)

In [104]:
tweets_lda_cm.show_topics()

[(0,
  '0.066*"prochoice" + 0.020*"prolife" + 0.018*"life" + 0.013*"woman" + 0.010*"abortion" + 0.009*"dont" + 0.009*"choice" + 0.008*"right" + 0.008*"like" + 0.008*"people"'),
 (1,
  '0.056*"prochoice" + 0.015*"choice" + 0.015*"abortion" + 0.011*"pro" + 0.009*"prolife" + 0.007*"life" + 0.007*"people" + 0.006*"amp" + 0.005*"th" + 0.005*"woman"'),
 (2,
  '0.047*"prochoice" + 0.013*"woman" + 0.011*"help" + 0.011*"abortion" + 0.009*"choice" + 0.009*"adoption" + 0.008*"many" + 0.008*"group" + 0.007*"amp" + 0.007*"would"'),
 (3,
  '0.070*"prochoice" + 0.021*"abortion" + 0.019*"woman" + 0.015*"prolife" + 0.011*"right" + 0.010*"u" + 0.007*"say" + 0.007*"one" + 0.006*"body" + 0.006*"like"'),
 (4,
  '0.055*"prochoice" + 0.019*"advocate" + 0.014*"human" + 0.013*"abortion" + 0.012*"woman" + 0.008*"science" + 0.008*"make" + 0.008*"unborn" + 0.008*"dont" + 0.008*"prominent"')]

In [105]:
cm = CoherenceModel(model=tweets_lda_cm, texts=df_cm.tok_tweet.to_list(), coherence='c_v')
cm.get_coherence()

0.3131768363188442

In [106]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(tweets_lda_cm, tweets_bow_cm, dictionary=txt_dict_cm)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.014840  0.043052       1        1  25.550314
3     -0.025373 -0.099549       2        1  23.992136
4      0.138025 -0.010554       3        1  23.049826
2     -0.086592 -0.022639       4        1  14.708044
1     -0.040900  0.089690       5        1  12.699681, topic_info=           Term       Freq      Total Category  logprob  loglift
1109   advocate  63.000000  63.000000  Default  30.0000  30.0000
7          life  82.000000  82.000000  Default  29.0000  29.0000
41         help  30.000000  30.000000  Default  28.0000  28.0000
69          pro  56.000000  56.000000  Default  27.0000  27.0000
1303  prominent  22.000000  22.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
12        think   4.845099  42.393905   Topic5  -5.7900  -0.1054
34         mean   3.972469  16.992165   Topic5  -5.9886   0.6102
124        like   4.970412  59.667741   Topic5  -5.7645  -0.4217
337       human   4.188956  64.951831   Topic5  -5.9355  -0.6776
37        right   4.081550  77.439373   Topic5  -5.9615  -0.8794

[375 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
40        1  0.185421  abortion
40        2  0.359605  abortion
40        3  0.213515  abortion
40        4  0.112377  abortion
40        5  0.129233  abortion
...     ...       ...       ...
3002      1  0.898692         x
1441      1  0.309172     youre
1441      2  0.077293     youre
1441      4  0.154586     youre
1441      5  0.386466     youre

[639 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 3, 2])